# MNIST Model Training
#### Written by Ervin Mamutov - github/imervin

### Introduction
This is my jupyter notebook containing the code I used to train my Tensorflow model to predict hand drawn figures for 808. The model is trained and tested using the MNIST dataset and built using Keras with Python 3.

I have adapted code from several places.

[1] https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

[2] http://parneetk.github.io/blog/cnn-mnist/

### What is MNIST?

MNIST(Modified National Institute of Standards and Technology) is a sub data set of NIST(National Institute of Standards and Technology), a large database of handwritten digits. MNIST is used to train image processing systems and is basically the "hello world" of machine learning and computer vision.

MNIST contains 60,000 training images and 10,000 testing images. Training images are used to train a system, and testing images are used to test the trained system.

### Where does the MNIST dataset come from?

The set of images in the MNIST database is a combination of two of NIST's databases: Special Database 1 and Special Database 3. Special Database 1 and Special Database 3 consist of digits written by high school students and employees of the United States Census Bureau, respectively.[1]


### What is Tensorflow and Keras?
Tensorflow is a popular software library for dataflow programming across a range of tasks. Tensorflow is open-source and is developed by the Google Brain Team. Tensorflow is a symbolic math library and is also used for machine learning applications such as neaural networks [2]. I will be using Tensorflow's Python API but it is available for a range of languages.

Keras is an open source neural network library written in Python developed by a Google engineer: Francois Chollet. Keras acts like a "library on top of a library" as it is capable of running on top of MXNet, Deeplearning4j, Tensorflow, CNTK or Theano. Keras takes the functionality in core Tensorflow and adds a higher-level of abstraction to it, making it easier to experiment with deep neural networks [3].

### 1. Download the MNIST dataset
Before I can start building my model, I must first get the MNIST dataset and decode it into a format that allows me to use it later on. Luckily MNIST is quite a popular dataset for machine learning and Keras comes with MNIST pre-built with the MNIST dataset.

The keras.datasets.mnist.load_data() produces 2 tuples:

    x_train, x_test: uint8 array of grayscale image data with shape (num_samples, 28, 28).
    y_train, y_test: uint8 array of digit labels (integers in range 0-9) with shape (num_samples,).

I will be renaming x_train, x_test to training_images, testing_images and y_train, y_test to training_labels and testing_labels but it will work the same as if I kept the names as x_train etc.

In [32]:
# Use keras's dataset mnist
from keras.datasets import mnist

# Initiate two tuples, the first with uint8 array of grayscale image data, the other with integers between 0-9
(training_images, training_labels), (testing_images, testing_labels) = mnist.load_data()

print("How many values in a training image?", len(training_images[7]) * len(training_images[7][0]))
print("What the 8th training image looks like:")
print(training_images[7])
print()
print("What the 8th training image looks like as a sequence of dots and hashes:")
# Visualize a training image by printing out it's RGB value as a #
for x in training_images[7]:
    print()
    for y in x:
        if(y != 0):
            print("#", end="")
        else:
            print(".", end="")

How many values in a training image? 784
What the 8th training image looks like:
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  38  43 105 255 253 253 253
  253 253 174   6   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  43 139 224 226 252 253 252 252 252
  252 252 252 158  14   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 178 252 252 252 252 253 252 252 252
  252 252 252 252  59   0 

As you can see, the array's values are displayed as integers between 0-255. There are 784 values and each value represents a pixel in the picture, the integer represents that pixel's RGB Grayscale value between 0-255. 0 means black, anything above 0 is lighter than black.

### 2. Import relevant libraries
I will need to import numpy to use numpys extensive arrays, keras to create, train and test the model

In [25]:
import keras as kr #importing keras
import numpy as np #importing numpy

### 3. Preparing the data for training/testing

In [47]:
print("Training/testing Images shape:",training_images.shape,"/",testing_images.shape)
print("Training/testing Labels shape:",training_labels.shape,"/",testing_labels.shape)
print("First 5 training labels and testing labels:", training_labels[:5], "/", testing_labels[:5])

Training/testing Images shape: (60000, 28, 28) / (10000, 28, 28)
Training/testing Labels shape: (60000,) / (10000,)
First 5 training labels and testing labels: [5 0 4 1 9] / [7 2 1 0 4]


As you can see from the output above, the shape of the image arrays is (number_of_images, 28, 28), meaning that there are 28 rows and 28 columns of pixels representing each picture. The shape of the label arrays is (number_of_labels), these labels correspond to the number_of_images in the image arrays. 

To prepare my data I must consider what type of architecture my neural network will have.. 
1. Basic Neural Network
2. Convolutional Neural Network

After some research I found that the basic NN works fine but limits out at around 96-97% accuracy and convolution neaural networks(CNN) hit around 99% accuracy using the same effort as you hit the point of deminishing returns sooner with an NN. [4] 

### Convolution

From what I understand **Convolution Neaural Networks** are made for image recognition and classification purposes, inspired by the animal visual cortex. CNN works by taking an image of n x n image (MyImage) and applying a k x k filter (or convolvution kernel) then compute MyImage x Convolution Kernel by multiplying the matrices to result in a new matrice that makes up the image. Once we have a new matrice, we add a "pooling" layer which will take a chunk of the image and aggregate them into a single value (downsampling) [5].

Here are image representations taken from [5].

Convolving the image matrice
![CNN1](https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/figures/convolve.png)
(I = MyImage and K = Convolution Kernel)

Maxpooling an chunk of the image
![CNN2](https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/figures/pool.png)

### Pre-processing arrays

As I'm building a CNN, I will need to reshape my data to add a "depth" dimension. A full image with all 3 RGB channels will have a depth of 3, however the mnist images only have a depth of 1 (grayscale). This means I must take the shape of the image arrays (number_of_images, 28, 28) and turn it into (number_of_images,1,28,28) [6]. Numpys .reshape function allows me to give an array a new shape without changing it's data. [7]

### References
[1] https://en.wikipedia.org/wiki/MNIST_database

[2] https://en.wikipedia.org/wiki/TensorFlow

[3] https://en.wikipedia.org/wiki/Keras

[4] https://datascience.stackexchange.com/questions/22173/why-not-use-more-than-3-hidden-layers-for-mnist-classification

[5] https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html

[6] https://elitedatascience.com/keras-tutorial-deep-learning-in-python

[7] https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.reshape.html